In [13]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1foGH2gugslHTYzKtG4Mw7VRaASEqgOwB' -O 'data.csv'

--2023-06-29 01:22:52--  https://drive.google.com/uc?export=download&id=1foGH2gugslHTYzKtG4Mw7VRaASEqgOwB
Resolving drive.google.com (drive.google.com)... 172.253.63.139, 172.253.63.102, 172.253.63.100, ...
Connecting to drive.google.com (drive.google.com)|172.253.63.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j3efdo0tnk2i0mc74m81cjshc4kftgc6/1688001750000/04531358851983181325/*/1foGH2gugslHTYzKtG4Mw7VRaASEqgOwB?e=download&uuid=0ef0cfd7-523e-42fe-96f4-d3f3450b5ed2 [following]
--2023-06-29 01:22:54--  https://doc-0k-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j3efdo0tnk2i0mc74m81cjshc4kftgc6/1688001750000/04531358851983181325/*/1foGH2gugslHTYzKtG4Mw7VRaASEqgOwB?e=download&uuid=0ef0cfd7-523e-42fe-96f4-d3f3450b5ed2
Resolving doc-0k-5g-docs.googleusercontent.com (doc-0k-5g-docs.googleusercontent.com)... 142.251.16.132, 2607:

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import sagemaker
tf.__version__

'2.12.0'

In [2]:
def extract(
    filename: str,  # name of the csv file
) -> pd.DataFrame:
    return pd.read_csv(filename)


loan_df = extract('data.csv')

In [3]:
def transform(
    dataframe: pd.DataFrame,
) -> pd.DataFrame:
    dataframe['zip_code'] = dataframe.address.apply(lambda x: x[-5:])

    dataframe.drop(['emp_title',
                    'emp_length',
                    'title', 'grade',
                    'address', 'issue_d', 'earliest_cr_line'], axis=1, inplace=True)
    total_acc_avg = dataframe.groupby(by='total_acc').mean().mort_acc

    def fill_mort_acc(total_acc, mort_acc):
        if np.isnan(mort_acc):
            return total_acc_avg[total_acc].round()
        else:
            return mort_acc

    dataframe['mort_acc'] = dataframe.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)
    dataframe.dropna(inplace=True)

    term_values = {' 36 months': 36, ' 60 months': 60}
    dataframe['term'] = dataframe.term.map(term_values)

    dummies = ['sub_grade', 'verification_status', 'purpose', 'initial_list_status', 'application_type', 'home_ownership', 'zip_code']
    dataframe = pd.get_dummies(dataframe, columns=dummies, drop_first=True)

    return dataframe

loan_df = transform(loan_df)

/tmp/ipykernel_10459/3909544060.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_acc_avg = dataframe.groupby(by='total_acc').mean().mort_acc


In [5]:
train, test = train_test_split(loan_df, test_size=0.33, random_state=42)

print(train.shape)
print(test.shape)

(264796, 79)
(130423, 79)


In [6]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [3]:
import boto3
sagemaker_session = sagemaker.Session()

In [4]:
#Uploading data to S3 bucket titled 
prefix = "tf-loan-delinq-data"
training_input_path = sagemaker_session.upload_data('train.csv', key_prefix=prefix + '/training')

In [5]:
training_input_path

's3://sagemaker-us-east-1-639966750355/tf-loan-delinq-data/training/train.csv'

In [6]:
#ensure training data has uploaded properly
training_data = pd.read_csv(training_input_path, sep =',')
training_data.head()

,Unnamed: 0,loan_amnt,term,int_rate,installment,annual_inc,loan_status,dti,open_acc,pub_rec,...,home_ownership_RENT,zip_code_05113,zip_code_11650,zip_code_22690,zip_code_29597,zip_code_30723,zip_code_48052,zip_code_70466,zip_code_86630,zip_code_93700
0,43729,6000.0,36,13.99,205.04,40000.0,1,24.39,12.0,0,...,1,0,0,0,0,0,0,1,0,0
1,300769,15000.0,36,7.62,467.43,120000.0,1,5.33,8.0,0,...,0,0,0,0,0,0,0,0,0,0
2,132630,2300.0,36,10.65,74.92,50000.0,1,6.62,3.0,0,...,1,0,0,0,0,1,0,0,0,0
3,114106,21000.0,60,15.61,506.34,225000.0,1,10.11,9.0,1,...,0,1,0,0,0,0,0,0,0,0
4,241171,15000.0,36,14.09,513.33,70000.0,1,18.22,6.0,0,...,1,1,0,0,0,0,0,0,0,0


In [7]:
#Sagemaker role, make sure you've allowed access to any S3 bucket
role = sagemaker.get_execution_role()
role

'arn:aws:iam::639966750355:role/service-role/AmazonSageMaker-ExecutionRole-20230628T125381'

In [8]:
#Need tf version and role for the Tensorflow Estimator
tf_version = tf.__version__
tf_version

'2.12.0'

In [19]:
#Use a tensorflow estimator from sagemaker to train model
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(
    entry_point='train.py',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version=tf_version,
    py_version='py310',
    script_mode=True,
    hyperparameters={
        'epochs': 20
    }
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [21]:
#Training
tf_estimator.fit({'train': training_input_path})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2023-06-29-03-36-18-741


Using provided s3_resource
2023-06-29 03:36:18 Starting - Starting the training job...
2023-06-29 03:36:36 Starting - Preparing the instances for training......
2023-06-29 03:37:31 Downloading - Downloading input data...
2023-06-29 03:38:11 Training - Downloading the training image........2023-06-29 03:39:25.923406: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 03:39:28,347 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2023-06-29 03:39:28,348 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-06-29 03:39:28,349 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-06-29 03:39:28,700 sagemaker-training-toolkit INFO     

In [22]:
#Creating endpoint and deploying model
import time
tf_endpoint_name = 'tf-loan-delinq-model'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
tf_predictor = tf_estimator.deploy(initial_instance_count=1,instance_type='ml.m5.large',
                                   endpoint_name=tf_endpoint_name)
print("Success!")

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: tensorflow-training-2023-06-29-03-50-39-237
INFO:sagemaker:Creating endpoint-config with name tf-loan-delinq-model2023-06-29-03-50-39
INFO:sagemaker:Creating endpoint with name tf-loan-delinq-model2023-06-29-03-50-39


---!Success!


In [25]:
predictions = tf_predictor.predict(
    [251650, 12000.0, 60, 19.69, 315.87, 63996.0, 0, 20.81, 6.0, 0, 22226.0,
     99.2, 38.0, 1.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
     0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
)['predictions']


def inferenceFunction(preds):
    preds = max(preds)
    maxValue = max(preds)
    maxIndex = preds.index(maxValue)
    if maxIndex == 0:
        print("Default")
    else:
        print("Paid")


results = inferenceFunction(predictions)
print(results)


Default
None


In [26]:
tf_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: tf-loan-delinq-model2023-06-29-03-50-39
INFO:sagemaker:Deleting endpoint with name: tf-loan-delinq-model2023-06-29-03-50-39
